In [18]:
# data_preprocess.py
from bs4 import BeautifulSoup
import re
import requests

ANEEL_RES_PATH = r"../data/ren20211000.html"
ANEEL_RES_PATH_2 = r"../data/ren20211000_2.html"
def fetch_html_content(url: str) -> str | None:
    """
    Faz uma requisição HTTP para obter o conteúdo HTML de uma URL.
    :param url: URL do qual o conteúdo HTML será obtido.
    :return: Conteúdo HTML como string.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Verifica se a requisição foi bem-sucedida
        
        # Get encoding from response headers or detect it
        if response.encoding is None or response.encoding == 'ISO-8859-1':
            # Try to detect encoding from content
            response.encoding = response.apparent_encoding
        
        # If still no proper encoding, default to utf-8
        if response.encoding is None:
            response.encoding = 'utf-8'
            
        return response.text
    except requests.RequestException as e:
        print(f"Erro ao obter conteúdo HTML de {url}: {e}")
        return None

def clean_html(html_path: str) -> str:
    """
    Lê um arquivo HTML, remove elementos com estilo `text-decoration: line-through` e extrai o texto limpo.
    : param html_path: Caminho para o arquivo HTML a ser processado.
    : return: Texto limpo extraído do HTML.
    """
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'lxml')
    
    for strike in soup.find_all(style=re.compile('line-through')):
        strike.decompose()
    
    # Extract clean text
    return soup.get_text(separator='\n', strip=True)

if __name__ == "__main__":
    cleaned_text = clean_html(ANEEL_RES_PATH)
    print("\n--- Cleaned Text (first 500 characters) ---")
    print(cleaned_text[:500])

    with open(r"../data/cleaned_text.txt", "w", encoding='utf-8') as output_file:
        output_file.write(cleaned_text)
    print("\nCleaned text saved to '../data/cleaned_text.txt'.")    


--- Cleaned Text (first 500 characters) ---
AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL
RESOLUÇÃO NORMATIVA ANEEL Nº 1.000, DE 7 DE DEZEMBRO DE 2021(*)
Estabelece as Regras de Prestação do Serviço Público de Distribuição de Energia Elétrica; revoga as Resoluções Normativas ANEEL nº
414
, de 9 de setembro de 2010; nº
470
, de 13 de dezembro de 2011; nº
901
, de 8 de dezembro de 2020 e dá outras providências.
Decisão Judicial
Despacho 2.006/2024: Decisão Judicial - suspensos os efeitos referentes ao prazo de 60 (sessenta) ciclos estabeleci

Cleaned text saved to '../data/cleaned_text.txt'.


In [19]:
html_content = fetch_html_content(r"https://www2.aneel.gov.br/cedoc/ren20211000.html")
if html_content:
    with open(ANEEL_RES_PATH_2, "w", encoding='utf-8') as file:
        file.write(html_content)
    print(f"HTML content fetched and saved to {ANEEL_RES_PATH_2}.")

HTML content fetched and saved to ../data/ren20211000_2.html.


In [20]:
html_content[:10000]

'<html>\r\n\t<head>\r\n\t\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n\t\t<meta http-equiv="Content-Style-Type" content="text/css" />\r\n\t\t<meta name="generator" content="Aspose.Words for .NET 21.2.0" />\r\n\t\t<title>\r\n\t\t</title>\r\n\t</head>\r\n\t<body style="font-family:\'Arial Narrow\'; font-size:12pt">\r\n\t\t<div>\r\n\t\t\t<p style="margin-top:0pt; margin-bottom:0pt; text-align:center">\r\n\t\t\t\t<span style="font-family:Calibri">AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL</span>\r\n\t\t\t</p>\r\n\t\t\t<p style="margin-top:0pt; margin-bottom:0pt; text-align:center">\r\n\t\t\t\t<span style="font-family:Calibri; -aw-import:ignore">&#xa0;</span>\r\n\t\t\t</p>\r\n\t\t\t<p style="margin-top:0pt; margin-bottom:0pt; text-align:center">\r\n\t\t\t\t<span style="font-family:Calibri; -aw-import:ignore">&#xa0;</span>\r\n\t\t\t</p>\r\n\t\t\t<p style="margin-top:0pt; margin-bottom:0pt; text-align:center">\r\n\t\t\t\t<span style="font-family:Calibri">RESOLU

In [2]:
# text_processor.py

from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(text: str, chunk_size: int=1000, chunk_overlap: int=200) -> list[str]:
    """
    Divide o texto em pedaços menores com base no tamanho e na sobreposição especificados.
    : param text: String de texto a ser dividido.
    : param chunk_size: Tamanho máximo de cada pedaço de texto.
    : param chunk_overlap: Número de caracteres que se sobrepõem entre pedaços consecutivos.
    : return: Lista de pedaços de texto divididos.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        add_start_index=True
    )
    documents = text_splitter.create_documents([text])
    return [doc.page_content for doc in documents]

if __name__ == "__main__":
    with open(r"../data/cleaned_text.txt", "r", encoding='utf-8') as file:
        cleaned_text = file.read()

    chunks = split_text(cleaned_text, chunk_size=1000, chunk_overlap=200)
    print("\n--- Text Chunks (first 3 chunks) ---")
    for i, chunk in enumerate(chunks[:3]):
        print(f"Chunk {i+1}:\n{chunk}\n")


--- Text Chunks (first 3 chunks) ---
Chunk 1:
AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL
RESOLUÇÃO NORMATIVA ANEEL Nº 1.000, DE 7 DE DEZEMBRO DE 2021(*)
Estabelece as Regras de Prestação do Serviço Público de Distribuição de Energia Elétrica; revoga as Resoluções Normativas ANEEL nº
414
, de 9 de setembro de 2010; nº
470
, de 13 de dezembro de 2011; nº
901
, de 8 de dezembro de 2020 e dá outras providências.
Decisão Judicial
Despacho 2.006/2024: Decisão Judicial - suspensos os efeitos referentes ao prazo de 60 (sessenta) ciclos estabelecidos no inciso II do art. 323
.
Voto
Texto Compilado
O DIRETOR-GERAL DA AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL, no uso de suas atribuições regimentais, de acordo com a deliberação da Diretoria, tendo em vista o disposto na Lei nº 9.427, de 26 de dezembro de 1996, no Decreto nº 2.335, de 6 de outubro de 1997 e o que consta do Processo nº 48500.005218/2020-06, resolve:
TÍTULO I
PARTE GERAL
CAPÍTULO I
DAS DISPOSIÇÕES GERAIS
Seção I
Do Objeto e Âmb

In [3]:
# vector_db.py

import chromadb
from chromadb.utils import embedding_functions

client  = None
collection = None
COLLECTION_NAME = "aneel_collection"

def initialize_vector_db(documents: list[str], persist_directory: str=r"../chroma_db_data"):
    """
    Inicializa o banco de dados vetorial ChromaDB com os documentos fornecidos.
    O embedding de documentos é feito usando o modelo 'all-MiniLM-L6-v2'.
    : param documents: Lista de documentos a serem armazenados no banco de dados.
    : param persist_directory: Diretório onde o banco de dados será persistido.
    : return: Instância do cliente ChromaDB e coleção criada.
    """
    global client, collection
    
    client = chromadb.PersistentClient(path=persist_directory)

    try:
        collection = client.get_collection(name=COLLECTION_NAME)
        print(f"Coleção '{COLLECTION_NAME}' carregada com sucesso.")
    except:
        print(f"Coleção '{COLLECTION_NAME}' não encontrada. Criando nova coleção.")
        collection = client.create_collection(name=COLLECTION_NAME)
        print(f"Coleção '{COLLECTION_NAME}' criada com sucesso.")

        doc_ids = [f"doc_{i}" for i in range(len(documents))]
        collection.add(
            documents=documents,
            ids=doc_ids,
            metadatas=[{"source": f"doc_{i}"} for i in range(len(documents))],
        )
        print(f"Banco de dados vetorial inicializado com {len(documents)} documentos.")
    
    return collection

def query_vector_db(query_text: str, n_results: int=5) -> list[str]:
    """
    Consulta o banco de dados vetorial ChromaDB com uma string de consulta.
    : param query_text: Texto da consulta para buscar documentos relevantes.
    : param n_results: Número de resultados a serem retornados.
    : return: Lista de IDs dos documentos mais relevantes encontrados.
    """
    global collection
    if not collection:
        print("Erro: Coleção não inicializada. Por favor, chame initialize_vector_db primeiro.")
        try:
            global client
            client = chromadb.PersistentClient(path=r"../chroma_db_data")
            collection = client.get_collection(name=COLLECTION_NAME)
            print(f"Coleção '{COLLECTION_NAME}' carregada com sucesso.")
        except Exception as e:
            print(f"Erro ao carregar a coleção: {e}")
            return []
    
    if not query_text:
        return []
    
    results = collection.query(
        query_texts=[query_text],
        n_results=n_results
    )
    return results['documents'][0]  if results and results['documents'] else []

if __name__ == "__main__":
    with open(r"../data/cleaned_text.txt", "r", encoding='utf-8') as file:
        cleaned_text = file.read()
    
    chunks = split_text(cleaned_text)
    initialize_vector_db(chunks)
    print("\n--- Vector Database Initialized ---")

    if collection:
        query = "Quais são as principais mudanças na regulamentação de energia?"
        print(f"\nQuery: {query}")
        retrieved_docs = query_vector_db(query, n_results=2)
        if retrieved_docs:
            print("\nRetrieved Documents:")
            for doc in retrieved_docs:
                print(f"- {doc[:300]}...")
        else:
            print("Nenhum documento encontrado para a consulta.")
    else:
        print("Falha ao inicializar a coleção. Verifique os logs para mais detalhes.")

Coleção 'aneel_collection' carregada com sucesso.

--- Vector Database Initialized ---

Query: Quais são as principais mudanças na regulamentação de energia?

Retrieved Documents:
- Seção III
Do Contrato de Compra de Energia Regulada – CCER
Art. 162. O CCER deve conter as cláusulas gerais do art. 145 e, caso aplicáveis, as do art. 132, além de outras consideradas essenciais, observando as demais disposições deste Capítulo.
Art. 163. O montante de energia elétrica contratado por...
- Art. 29. O consumidor e demais usuários devem observar em suas instalações as normas e padrões da distribuidora, as normas da Associação Brasileira de Normas Técnicas - ABNT e as normas dos órgãos oficiais competentes, naquilo que for aplicável e não contrariar à regulação da ANEEL.
Art. 30. O consu...


General text: 500-1000 characters

Technical docs: 300-800 characters

Code: 200-500 characters (split at logical boundaries)

In [4]:
retrieved_docs = query_vector_db("Qual é o periodo de testes para unidade consumidara?", n_results=2)
if retrieved_docs:
    print("\nRetrieved Documents:")
    for doc in retrieved_docs:
        print(f"- {doc}...")


Retrieved Documents:
- Seção XI
Do Período de Testes e Ajustes
Art. 311. A distribuidora deve aplicar o período de testes para unidade consumidora para permitir a adequação da demanda contratada de consumo e a escolha da modalidade tarifária, nas seguintes situações: (
Redação dada pela REN ANEEL 1.059, de 07.02.2023
)
I - início do fornecimento de energia elétrica;
II - mudança para faturamento aplicável à unidade consumidora do grupo
A, cuja opção anterior tenha sido por faturamento do grupo B;
III - enquadramento na modalidade tarifária horária azul; e
IV - acréscimo de demanda, quando maior que 5% da contratada.
Parágrafo único. Quando do enquadramento na modalidade tarifária horária azul, o período de testes abrangerá exclusivamente o montante contratado para o posto tarifário ponta.
Art. 312. O período de testes deve ter duração de 3 ciclos consecutivos e completos de faturamento.
Parágrafo único.
A distribuidora pode prorrogar o período de testes, mediante solicitação fundament

In [ ]:
# chatbot_logic.py
import google.generativeai as genai
import os

from dotenv import load_dotenv
load_dotenv()

try:
    if not os.getenv("GOOGLE_API_KEY"):
        raise ValueError("A variável de ambiente GOOGLE_API_KEY não está definida.")
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
except ValueError as e:
    print(f"Erro de configuração: {e}. Garanta que a variável de ambiente GOOGLE_API_KEY esteja definida.")

def generate_response_with_gemini(query: str, context_chuncks: list[str]) -> str:
    """
    Gera uma resposta usando o modelo Gemini da Google Generative AI, incorporando o contexto dos documentos recuperados.
    : param query: Texto da consulta do usuário.
    : param context_chuncks: Lista de pedaços de texto que fornecem contexto adicional para a resposta.
    : return: Resposta gerada pelo modelo Gemini.
    """
    if not os.getenv("GOOGLE_API_KEY"):
        raise ValueError("A variável de ambiente GOOGLE_API_KEY não está definida.")
    
    # Prepara o contexto para a consulta
    context_str = "\n\n---\n\n".join(context_chuncks)
    prompt = f"""
    Você é um assistente de IA especializado em leis e regulamentos brasileiros da ANEEL.
    Sua tarefa é responder à pergunta do usuário com base estritamente nos trechos de lei fornecidos abaixo.
    Não utilize conhecimento externo. Se a resposta não puder ser encontrada nos trechos fornecidos,
    declare explicitamente que a informação não está disponível nos documentos consultados.
    Seja conciso e direto ao ponto. Responda em português brasileiro.
    
    **Contexto (Trechos da Lei):**
    {context_str}
    
    **Pergunta do Usuário:**
    {query}

**Sua Resposta:**
"""
    try:
        model = genai.GenerativeModel(model_name="gemini-1.5-flash-8b")
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Erro ao gerar resposta com Gemini: {e}")
        return "Desculpe, ocorreu um erro ao processar sua consulta. Tente novamente mais tarde."

test_user_query = "O que deve constar na segunda via de fatura?"
retrieved_docs = query_vector_db(test_user_query, n_results=2)
if retrieved_docs:
    print("\nRetrieved Documents for Test Query:")
    for doc in retrieved_docs:
        print(f"- {doc[:300]}...")

    response = generate_response_with_gemini(test_user_query, retrieved_docs)
    print("\n--- Response from Gemini ---")
    print(response)


Retrieved Documents for Test Query:
- § 3º A distribuidora deve disponibilizar a fatura de energia elétrica completa no espaço reservado de atendimento pela internet, independentemente da opção pelo resumo da fatura.
Art. 329. A distribuidora deve prestar ao consumidor e demais usuários esclarecimentos sobre os tributos, subvenções e in...
- VII - formalidades e exigências que sejam incompatíveis com a boa-fé, excessivamente onerosas ou cujo custo econômico ou social seja superior ao risco envolvido.
Parágrafo único. No caso de núcleo urbano informal consolidado, nos termos da Lei nº 13.465, de 11 de julho de 2017, a comprovação de poss...

--- Response from Gemini ---
Todas as informações da primeira via, com destaque à expressão "segunda via".

